# 1. Import Libaries

In [1]:
import os
import shutil
import sys
import numpy as np
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as dset
from torchvision import transforms
from torch import autograd
import torch.optim as optim
import torchvision.utils as vutils # draw bounding box, segmantation mask, keypoints. convert to rgb, make grid, save_image

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# import ignite
# import ignite.distributed as idist
# from ignite.metrics import FID, InceptionScore, RunningAverage
# from ignite.contrib.handlers import ProgressBar
# from ignite.engine import Engine, Events
# from ignite.handlers import *
import PIL.Image as Image

from src.utils import geometric_score as gs

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
DATAROOT = "./data/cadastralExportRGB/train/"  # root directory for dataset
MODEL = "dcgan_embeddings"
WORKERS = 4  # number of workers for dataloader
BATCH_SIZE = 2000  # batch size during training
IMG_SIZE = 300  # spatial size of training images (to be resized to)
MULT = 3.15  # re-size factor: 11 if resolution is 64 x 64, 3.15 if resolution is 300 x 300
NC = 3  # number of entities aka channels in the training images
NZ = 512  # size of noise vector (i.e. size of generator input)
NGF = 32  # base size of feature maps in generator
NDF = 32  # base size of feature maps in discriminator
NUM_EPOCHS = 1000  # number of training epochs
LR = 1e-2  # learning rate for both optimizers
DEVICE = 'cpu'
# DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")  # which device to run on

In [3]:
x_true = torch.load('./data/test.pkl')
dataloader = torch.utils.data.DataLoader(x_true, batch_size=BATCH_SIZE, 
                                             shuffle=True, num_workers=4, drop_last=True)
x_true = next(iter(dataloader))
x_true = x_true[0].to(DEVICE)

In [4]:
netG = torch.load(f'./models/netG_{MODEL}.pkl').to(DEVICE)
netD = torch.load(f'./models/netD_{MODEL}.pkl').to(DEVICE)
fixed_noise = torch.randn(2000, NZ, 1, 1, device=DEVICE)
x_pred = netG(fixed_noise)
y_pred = netD(x_pred)
y_true = netD(x_true)

In [5]:
y_pred.shape

torch.Size([2000, 1, 32, 32])

Random generator

In [4]:
class Generator(nn.Module):
    def __init__(self, nz, nc, ngf, drop_rate):
        super(Generator, self).__init__()
        self.softmax = nn.Softmax(dim=1)
        self.main = nn.Sequential(
            nn.ConvTranspose2d(in_channels=nz, out_channels=ngf * 8, kernel_size=2, stride=1, 
                               padding=0, bias=False),
            nn.Dropout2d(p=drop_rate),
            nn.BatchNorm2d(ngf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(in_channels=ngf * 8, out_channels=ngf * 4, kernel_size=4, stride=2, 
                               padding=0, bias=False),
            nn.Dropout2d(p=drop_rate),
            nn.BatchNorm2d(ngf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(in_channels=ngf * 4, out_channels=ngf * 2, kernel_size=5, stride=3, 
                               padding=1, bias=False),
            nn.Dropout2d(p=drop_rate),
            nn.BatchNorm2d(ngf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(in_channels=ngf * 2, out_channels=ngf, kernel_size=6, stride=4, 
                               padding=0, bias=False),
            nn.Dropout2d(p=drop_rate),
            nn.BatchNorm2d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(in_channels=ngf, out_channels=nc, kernel_size=12, stride=4, 
                               padding=2, bias=False)
        )
    
    def forward(self, input_):
        input_ = self.main(input_)
        return self.softmax(input_)
    
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [5]:
# Create the generator
netG = Generator(NZ, NC, NGF, 0.05).to(DEVICE)
# netG = torch.load('./models/netG.pkl').to('cpu')

# # Handle multi-GPU if desired
# if (device.type == 'cuda') and (ngpu > 1):
#     netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (softmax): Softmax(dim=1)
  (main): Sequential(
    (0): ConvTranspose2d(100, 256, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (1): Dropout2d(p=0.05, inplace=False)
    (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (5): Dropout2d(p=0.05, inplace=False)
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1), bias=False)
    (9): Dropout2d(p=0.05, inplace=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2, inplace=True)
    (12): ConvTranspose2d(64, 32, kernel_size=(6, 6), stride=(4, 4), bias=False)
    (13): Dropout2d(p=0.05, inplace=False

In [7]:
fixed_noise = torch.randn(2000, NZ, 1, 1, device=DEVICE)
x_pred = netG(fixed_noise)

# Calculating Geometric Score

Implementation: https://github.com/KhrulkovV/geometry-score

In [ ]:
geom_score = []
for i in [0, 2]:
    rltx = gs.rlts(x_true[:, i, :, :].view(BATCH_SIZE, -1).numpy(), n=100, L_0=32, i_max=10, gamma=1.0/8)
    # print(f"Score for {i}-th layer in true images: {rltx}")
    rlty = gs.rlts(x_pred[:, i, :, :].view(BATCH_SIZE, -1).cpu().detach().numpy(), n=100, L_0=32, i_max=10, gamma=1.0/8)
    # print(f"Score for {i}-th layer in fake images: {rlty}")
    geom_score.append(gs.geom_score(rltx, rlty))
local_score = sum(geom_score) / 2
rltx = gs.rlts(x_true[:, 1, :, :].view(BATCH_SIZE, -1).numpy(), n=100, L_0=32, i_max=10, gamma=1.0/8)
# print(f"Score for {i}-th layer in true images: {rltx}")
rlty = gs.rlts(x_pred[:, 1, :, :].view(BATCH_SIZE, -1).cpu().detach().numpy(), n=100, L_0=32, i_max=10, gamma=1.0/8)
# print(f"Score for {i}-th layer in fake images: {rlty}")
global_score = gs.geom_score(rltx, rlty)
total_score = (local_score + global_score) / 2

# t-SNE of embeddings

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(x) 